In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets

In [2]:
digits = datasets.load_digits()

In [3]:
X = digits.data
y = digits.target

In [4]:
X.shape

(1797, 64)

In [5]:
y.shape

(1797,)

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.4,random_state=666)

In [8]:
from sklearn.neighbors import KNeighborsClassifier

In [9]:
best_score,best_k,best_p = 0,0,0

In [10]:
for k in range(2,11):
    for p in range(1,6):
        knf = KNeighborsClassifier(n_neighbors=k,p=p,weights="distance")
        knf.fit(X_train,y_train)
        
        score = knf.score(X_test,y_test)
        if score > best_score:
            best_score,best_k,best_p = score,k,p
print("best_k:",best_k)
print("best_p:",best_p)
print("best_score:",best_score)

best_k: 3
best_p: 4
best_score: 0.9860917941585535


### 使用交叉验证

In [11]:
from sklearn.model_selection import cross_val_score

In [12]:
knn_clf = KNeighborsClassifier()
cross_val_score(knn_clf,X_train,y_train)

array([0.98895028, 0.97777778, 0.96629213])

In [13]:
best_score,best_k,best_p = 0,0,0
for k in range(2,11):
    for p in range(1,6):
        knf = KNeighborsClassifier(n_neighbors=k,p=p,weights="distance")
        knf.fit(X_train,y_train)
        score = cross_val_score(knf,X_train,y_train)
        score = np.mean(score)
        if score > best_score:
            best_score,best_k,best_p = score,k,p
print("best_k:",best_k)
print("best_p:",best_p)
print("best_score:",best_score)

best_k: 2
best_p: 2
best_score: 0.9823599874006478


In [14]:
best_knf = KNeighborsClassifier(n_neighbors=2,p=2,weights="distance")
best_knf.fit(X_train,y_train)
best_knf.score(X_test,y_test)

0.980528511821975

### 回顾网格搜索

In [15]:
from sklearn.model_selection import GridSearchCV

In [16]:
param_grid = [
    {
        'weights':['distance'],
        'n_neighbors':[i for i in range(2,11)],
        'p':[i for i in range(1,6)]
    }
]

grid_search = GridSearchCV(knn_clf,param_grid,verbose=1)
grid_search.fit(X_train,y_train)

Fitting 3 folds for each of 45 candidates, totalling 135 fits


[Parallel(n_jobs=1)]: Done 135 out of 135 | elapsed:  1.3min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'weights': ['distance'], 'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10], 'p': [1, 2, 3, 4, 5]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [17]:
grid_search.best_score_

0.9823747680890538

In [18]:
grid_search.best_params_

{'n_neighbors': 2, 'p': 2, 'weights': 'distance'}

In [19]:
best_knn_clf = grid_search.best_estimator_
best_knn_clf.score(X_test,y_test)

0.980528511821975

In [20]:
cross_val_score(knn_clf,X_train,y_train,cv=5)

array([0.99543379, 0.97716895, 0.97685185, 0.98130841, 0.97142857])

In [21]:
GridSearchCV(knn_clf,param_grid,verbose=1,cv=5)

GridSearchCV(cv=5, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'weights': ['distance'], 'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10], 'p': [1, 2, 3, 4, 5]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)